In [13]:
from pyiem.util import get_dbconn
from pyiem.plot import get_cmap
from pandas.io.sql import read_sql
from pyiem.plot import MapPlot
import matplotlib.cm as cm

In [29]:
obs = read_sql(
    """
    select id, ST_x(geom) as lon, st_y(geom) as lat, state, climate_site,
    sum(case when max_tmpf is not null then 1 else 0 end) as days,
    sum(case when max_tmpf >= 89.5 then 1 else 0 end) as hits from
    summary_2021 s JOIN stations t on (s.iemid = t.iemid) WHERE t.network ~* 'ASOS' and
    t.country = 'US' and climate_site is not null
    GROUP by id, lon, lat, state, climate_site ORDER by climate_site ASC
    """,
    get_dbconn("iem"),
    index_col='id',
)
obs = obs[obs["days"] > 235]
obs["climo"] = -1

In [30]:
cursor = get_dbconn("coop").cursor()
for sid, row in obs[obs["climo"] < 0].iterrows():
    clsite = row["climate_site"]
    cursor.execute(f"""
    with obs as (
        select year, count(*), sum(case when high >= 90 then 1 else 0 end) as events from
        alldata_{clsite[:2]} where station = %s and day >= '1951-01-01' and sday < '0826' GROUP by year
    )
    select avg(events), count(*) from obs where count > 235
    """, (clsite, ))
    entry = cursor.fetchone()
    if entry[1] < 50:
        print(f"skip {clsite} {entry}")
        continue
    obs.at[sid, 'climo'] = entry[0]
obs

skip AR5320 (Decimal('54.4782608695652174'), 46)
skip AR5320 (Decimal('54.4782608695652174'), 46)
skip CA0192 (Decimal('25.6875000000000000'), 32)
skip CA2574 (Decimal('43.3720930232558140'), 43)
skip CA2706 (Decimal('29.1190476190476190'), 42)
skip CA4749 (Decimal('74.4468085106382979'), 47)
skip CA4881 (Decimal('7.8787878787878788'), 33)
skip CATRNM (Decimal('37.7872340425531915'), 47)
skip CO0950 (Decimal('37.8750000000000000'), 48)
skip CO1932 (Decimal('13.0454545454545455'), 44)
skip CO5236 (Decimal('30.4375000000000000'), 32)
skip CO5984 (Decimal('41.1621621621621622'), 37)
skip FL3874 (Decimal('58.4772727272727273'), 44)
skip ID7644 (Decimal('14.5454545454545455'), 44)
skip MA4502 (Decimal('8.0000000000000000'), 37)
skip MD2282 (Decimal('30.2978723404255319'), 47)
skip MS9426 (Decimal('40.7435897435897436'), 39)
skip MS9426 (Decimal('40.7435897435897436'), 39)
skip NC0982 (Decimal('0.68292682926829268293'), 41)
skip NC7656 (Decimal('46.7142857142857143'), 49)
skip NM2250 (Decima

,lon,lat,state,climate_site,days,hits,climo
id,,,,,,,
PAJN,-134.57628,58.35497,AK,AK0464,237,0,0
PAKT,-131.71167,55.35667,AK,AK2227,237,0,0
PAOR,-141.92917,62.96139,AK,AK2607,237,0,0
PAFA,-147.87611,64.80389,AK,AK3275,237,0,0
PAKU,-149.59750,70.33083,AK,AK3275,237,0,0
...,...,...,...,...,...,...,...
RKS,-109.06519,41.59422,WY,WY7845,237,25,7
SHR,-106.97000,44.77000,WY,WY8155,237,53,26
HSG,-108.38970,43.71360,WY,WY8875,237,44,35


In [34]:

obs["state"] = obs["climate_site"].str.slice(0, 2)
obs["delta"] = obs["hits"] - obs["climo"]

In [57]:
obs[obs["state"] == "AZ"].sort_values(by="delta")

,lon,lat,state,climate_site,days,hits,climo,delta
id,,,,,,,,
SEZ,-111.788444,34.848583,AZ,AZ5635,237.0,71.0,96.0,-25.0
PAN,-111.339256,34.256836,AZ,AZ6323,237.0,37.0,53.0,-16.0
OLS,-110.845830,31.420830,AZ,AZ8865,237.0,78.0,92.0,-14.0
RQE,-109.061390,35.657500,AZ,AZ7488,237.0,34.0,40.0,-6.0
SOW,-110.005670,34.265470,AZ,AZ7855,236.0,20.0,18.0,2.0
PRC,-112.420000,34.650000,AZ,AZ6796,237.0,36.0,32.0,4.0
FFZ,-111.728330,33.460830,AZ,AZ8499,237.0,123.0,119.0,4.0
FLG,-111.672220,35.140280,AZ,AZ3010,237.0,10.0,3.0,7.0
A39,-111.918500,32.990800,AZ,AZ1306,237.0,125.0,117.0,8.0


In [58]:
for sid in ['PGA', 'GCN', 'APC', 'IPL', 'NXP', 'L08', 'PSP', 'TRM', 'AAT', 'SDB', 'NYL', 'SDL', '3S8', 'SUT', 'SEZ']:
    obs.at[sid, 'climo'] = -1
obs = obs[obs["climo"] > -1].copy()

In [60]:
cmap = get_cmap('RdBu_r')
cmap.set_under('black')
cmap.set_over('pink')
m = MapPlot(sector='conus', title="1 Jan - 25 Aug 2021 Departure of Number of 90+$^\circ$F High Temp Days",
            subtitle='airport weather stations vs COOP 1951-2020 average number of days to date')
m.contourf(obs["lon"], obs["lat"], obs["hits"] - obs["climo"], range(-25,26,5) , cmap=cmap, units='days')
#m.plot_values(lons, lats, labels, '%s')
m.postprocess(filename='210826.png')